# Session 3.2: Detector Paramaters

## Aims of this session
- Introduction to more advanced detector properties
- Learn the effect of magnification on tube-based scanners
- See how the detector's Line Spread Function dramatically change result quality (pixels are not everything!)

In [ ]:
from gvxrPython3 import gvxr, json2gvxr
import matplotlib.pyplot as plt
import matplotlib.ticker as plttick
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
from math import e, pi, sqrt
from scipy.signal import unit_impulse

# Load in gvxr paramaters from a json file
json2gvxr.initGVXR("JSON/notebook5-detector-lsf.json")
json2gvxr.initDetector()
json2gvxr.initSourceGeometry()
json2gvxr.initSpectrum()
json2gvxr.initSamples()

# Remove the current LSF
gvxr.setLSF((0,1,0))

# Introduction to the Detector
The Detector is a core component of all scanning systems. It is the part recieving X-rays from the beam source, converting them into signals (the images we see).

In a previous setting you quickly looked at defining a detector's spacial location, along with the number and size of pixels the detector will recieve. In this notebook, we will take a further in-depth look into how these paramaters are not enough to accurately simulate real-life detectors.

In GVXR, the detector is modelled as a simple 2D plane in the path of the beam source. This virtual detector by default has perfect properties, resulting in a theoretical perfect image. Real life detectors are unfortunately not perfect, and obtaining high quality images relies on more than resolution.

![](img/scene-detector.drawio.png)


Just like a camera, detectors can be built in many different ways. One common detector type uses scintillators to absorb x-rays and re-emit the absorbed energy as light. This light is then detected using transistors to produce an electrical image.

![](img/detector-scintillator.drawio.png)

Keep in mind that this process isn't perfect! the generated light energy is picked up by neighbouring transistors, causing blurring. Depending on the quality of the detector, this effect can drastically effect the quality of the produced scan. This phenomenon can be measured and used to calibrate detectors. The measured response is called the Line Spread Function.


## Resolution and Pixel Size

Detectors come in many different sizes, and are represented in different formats.
GVXR only supports denoting detector sizes using pixel size, and resolution.

Sometimes, the term *pixel pitch* is used. Pixel pitch is the distance between the centre of two pixels. It is often assumed that the edge of pixels have no width;- resulting in a pixel size equal to the pixel pitch.

![](img/detector-size.drawio.png)

### Task: Setting Pixel Size and Resolution

Setting the detector size in gvxr is easy, but it only supports pixel values.
The detector's resolution *must* be in pixels, you will have to convert the values yourself.

In [ ]:
# Set resolution to 512x512 with a pixel size of 200μm
gvxr.setDetectorNumberOfPixels(512, 512)
gvxr.setDetectorPixelSize(200, 200, "um")

# Create a detector with a width of 8.192mm, height of 12.046mm and with a pixel size of 8μm
gvxr.setDetectorPixelSize(8, 8, "um")
gvxr.setDetectorNumberOfPixels(1024, 1506)

# Using a resolution of 500x1000, create a detector with a width of 10mm and height of 20mm
gvxr.setDetectorNumberOfPixels(500, 1000)
gvxr.setDetectorPixelSize(20,20,"um")

# Create a detector with a resolution of 512x512 with a pixel pitch of 20μm
gvxr.setDetectorNumberOfPixels(512, 512)
gvxr.setDetectorPixelSize(20,20,"um")


### Task: Pixel Stride

Pixel stride is a method to reduce noise by combining values from multiple pixels and resulting in a lower resultion image.
The result may have less total resolution, but the overall noise is decreased as more samples per pixel are used.

![](img/detector-stride.drawio.png)

GVXR does not directly support detector stride, however there are two approaches to recreate this detector feature:
1. Double the pixel size to directly reduce the resolution
2. Sample a full-resolution image

Method 2. can be done using [scikit-learn's `downscale_local_mean`](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.downscale_local_mean) but that will not be convered in this lesson.

In [ ]:
# Approach 1 - Doubling pixel size

# Create a detector with a width of 100cm, height of 120cm and with a pixel size of 80μm
gvxr.setDetectorPixelSize(80,80,"um")
gvxr.setDetectorNumberOfPixels(1250,1500)

refImg = gvxr.computeXRayImage()

# Create the same detector, but with a pixel size of 160μm to recreate a pixel stride of 2
gvxr.setDetectorPixelSize(160,160,"um")
gvxr.setDetectorNumberOfPixels(625, 750)

newImg = gvxr.computeXRayImage()

# -----
# Make a simple side-by-side with matplotlib
fig, axes = plt.subplots(1,2)
axes[0].imshow(refImg,cmap="gray")
axes[1].imshow(newImg,cmap="gray")

Here we can see that the physical pane size has not changed, but the resolution is halved.

## OTF, MTF, PSF, ESF, and LSF

OTF, MTF, PSF, and LSF are all values used to represent the 'spillover' caused by imperfect detectors. Since scintillator detectors are not perfect there is light leakage causing other nearby pixels to be effected. This has a dramatic effect on scans. Although the large number of acronyms can be confusing, we are only interested in LSF and PSF.

The Optical Transfer Function (OTF) specifies how the detector system reacts to different spacial frequencies. It describes in the Fourier domain the transform of the Point Spread Function (PSF) applied to an image recieved by the detector. The Modulation Transfer Function (MTF) is defined as the magnitude of the OTF and has similar uses in different fields.

![OTF relationship](img/otf.drawio.png)

In our specific case, we are interested in the Point Spread Fuction (PSF) and Line Spread Function (LSF).
The PSF can be seen as the effect a single pixel has on the surrounding pixels, whereas LSF is a single line of this PSF though the origin. As mentioned previously in this notebook, a detector's consutrction is not perfect, and the scintillators commonly used in detectors are picked up by neighbouring pixels.

![](img/detector-scintillator.drawio.png)

LSF is used rather than PSF as the measurement can be calculated from an image by using a sharp gradient change, and calculating the Edge Spread Function (ESF). The first derivative of the ESF is the LSF.
Although it would be more mathematically correct to use the PSF; this cannot be calculated using experiments, as the original obtained LSF is already been affected by the vertical component of the PSF.
This is regarded as generally a non-issue, since the common usage of xCT is to assume slices are independent of each other.

If the above explanation has confused you, don't worry. The key influenceing factor on scans is the Line Spread Function, as it is responsible for blurring in images.

![Detector LSF](img/detector-lsf.png)

Shown below are two images; one of a Siemens Star with no LSF applied, and another with a LSF calibrated from a physical detector

| Perfect Detector | Real Detector |
| --- | --- |
| ![](img/siemens-perfect.png) | ![](img/siemens-real.png) |

Notice how even with the same detector paramaters, the image is heavily blurred.
Another example with the same setup, but taken as a CT acquisition and reconstruction:

| Plate CT Perfect Detector | Plate CT Real Detector |
| --- | --- |
| ![](img/plate-recon-perfect.png) | ![](img/plate-recon-real.png) |

<small>(Reconstructed using 360 projections with FDK)</small>

### LSF Playground

Now that you know what the LSF is, this is a small demo where you can adjust the 'Spikiness' of an LSF function.

Use the slider to adjust the peak size, and click simulate to generate a quick projection. Feel free to look at the code and see how it works!

In [ ]:
%matplotlib widget

# Widgets
slider = widgets.FloatSlider(value=0.9,min=0.1,max=0.90,step=0.01,description="LSF Peak",readout=True)
button = widgets.Button(description="Simulate")
color = widgets.Checkbox(description="Grayscale")
output = widgets.Output(layout=widgets.Layout(height='600px'))

# Simulate LSF
def simulate_lsf(b):
    with output:
        # Create a Dirac
        a = slider.value * -1 + 1
        x = np.linspace(-2,2,41)
        lsf = ((1 / abs(a) * sqrt(pi)) * e) ** (((x / a) ** 2) * -1)
        lsf /= np.sum(lsf)

        # Simulate Projection
        gvxr.setLSF(lsf)
        image = gvxr.computeXRayImage()

        # Plot LSF
        output.clear_output()
        fig, axes = plt.subplots(1,2)
        axes[0].set_title("Detector LSF")
        axes[0].set_xlabel("Pixel")
        axes[0].set_ylabel("Pixel Contribution")
        axes[0].set_ybound(0,1)
        axes[0].set_xbound(-20,20)
        axes[0].set_autoscale_on(False)
        axes[0].plot(np.linspace(-20,20,41), lsf)
        cmap = "gray" if color.value else None
        axes[1].imshow(image,cmap=cmap)
        plt.show()

# Bind and display
button.on_click(simulate_lsf)
display(output)
display(widgets.HBox((button,slider,color)))
simulate_lsf(None)

## Flatfield Correction

So far, apart from scintillator paramaters, we've been assuming the detector itself is mostly perfect. In the real case, this is incorrect.
The detector can build up defects over time, such as dust, and the beam itself is not a uniform spread of intensity across the detector.
Flatfield Correction is an approach to remove this fixed-pattern noise, and reduce image quality loss.

In order to overcome this, scanners use flatfields as a way to calibrate and correct scanners to ensure a high quality image.
A lightfield is a scan image with no sample in place, and represents the maximum beam intensity for each pixel.
This lightfield is then used with a darkfield;- an image where the beam is off and represents the minimum intensity of each pixel.

$$ corrected = \frac{raw - darkfield}{lightfield - darkfield} $$


Below is the lightfield of our simulated scanner;- it is a uniform white image as every pixel recieves the same beam intensity.
Next to it is a lightfield from a beamline at the Diamond Light Source taken from a recent experiment.

|Ideal Lightfield | Real Lightfield |
| --- | --- |
| ![](img/lightfield-simulated.png)|![](img/lightfield-diamond.png)|

In industral machines this process is automatic, and result images have already been normalized. In more specialised scenarios such as synchrotrons and some lab systems, this has to be done manually. In simulations, the effect of flatfield correction is neglible on the final result, and therefore when using GVXR we assume the detector is perfect in this sense.

*Note: flatfield correction still has to be done in order to balance out photon energy, even if no fixed-pattern noise is present!*


# Energy Response

Another often overlooked property of a detector is its response to different energies. As with all optical systems, X-ray detectors do not respond equally to all energies.
This behaviour is not linear, and will effect the values of a recovered material's mu. However this non-linear relationship can be solved by assuming a lower or upper energy bound based on the scanner and experiment.

In [ ]:
# Read response data
energy_response = np.loadtxt("input_data/responseDetector.txt")[:200] * 1000
fig, ax = plt.subplots()
plt.plot(energy_response[:,0], energy_response[:,1])
plt.title("Detector Energy Response")
plt.xlabel("Power Input (keV)")
plt.ylabel("Power Output (keV)")

As seen in the graph above, a recieved power input of 40keV will result in a 40keV output. However, an input of 104keV will also show 40keV!